In [ ]:
ht_model = joblib.load('height_model.pkl')
wt_model = joblib.load('width_model.pkl')
dt_model = joblib.load('depth_model.pkl')
generic_model = joblib.load('generic_model.pkl')

In [19]:
def get_model(entity_name):
    model = None
    
    match entity_name:
        case "height":
            return ht_model
        case "width":
            return wt_model
        case "depth":
            return dt_model
        case default:
            return generic_model

In [16]:
def get_probability_from_model(columns, entity_name):
    columns = normalize_columns(columns)
    
    # Load the saved model
    model = get_model(entity_name)

    feature_names = ['x_min', 'y_min', 'height', 'width', 'zero', 'plus_90', 'minus_90']

    # Convert the list to a DataFrame with correct feature names
    features = pd.DataFrame([columns], columns=feature_names)

    predicted_proba = model.predict_proba(features)
    return predicted_proba[0]

In [3]:
def normalize_columns(columns):
    
    scaling_factors = {
     'x_min': {'mean': np.float64(579.5818181818182), 'std': np.float64(428.7543841006432)},
     'y_min': {'mean': np.float64(1007.4545454545455), 'std': np.float64(521.0780782535028)}, 
     'height': {'mean': np.float64(40.836363636363636), 'std': np.float64(35.57764780826612)},
     'width': {'mean': np.float64(46.345454545454544), 'std': np.float64(41.67476284978546)}
    }

    new_columns = {
        'x_min':(columns['x_min'] - scaling_factors['x_min']['mean'])/scaling_factors['x_min']['std'],
        'y_min':(columns['y_min'] - scaling_factors['y_min']['mean'])/scaling_factors['y_min']['std'],
        'height':(columns['height'] - scaling_factors['height']['mean'])/scaling_factors['height']['std'],
        'width':(columns['width'] - scaling_factors['width']['mean'])/scaling_factors['width']['std'],
        'zero':columns['zero'],
        'plus_90':columns['plus_90'],
        'minus_90':columns['minus_90']
    }

    return new_columns

In [2]:
def expand_orientation(orientation):

    if orientation[-2] == '_':
        # zero
        return 1,0,0
    elif orientation[-4] == 'g':
        # neg
        return 0,0,1
    else:
        # pos
        return 0,1,0

In [ ]:
## format:

## metadata_value x_min y_min height width orientation
## 200mg           600    400    45    67     ocr_results_0 

In [ ]:
def get_paddle_data(image_link, category_id, entity_name):
    
    #call whatever the fuck you want to and get me the data
    return 42

In [ ]:
def get_probabilities(image_link, category_id, entity_name):
    
    option_df = get_paddle_data(image_link, category_id, entity_name)
    options = []
    for index, row in option_df.iterrows():
        
        zero, plus_90, minus_90 = expand_orientation(rows['orientation'])
        
        word = row['metadata_value']
        columns = {
            'x_min':row['x_min'],
            'y_min':row['y_min'],
            'height':row['width'],
            'width':row['height'],
            'zero':zero,
            'plus_90':plus_90,
            'minus_90':minus_90
        }

        prob = get_probability_from_model(columns, entity_name)
        options.append({'word':word,'prob':prob[1]})
    return options

In [ ]:
def predictor(image_link, category_id, entity_name):
    a = get_probabilities(image_link, category_id, entity_name)
    max_prob = -1
    best_guess = ""
    for values in a:
        if(values['prob']>max_prob):
            max_prob = values['prob']
            best_guess = values['word']
    print best_guess